# Vicroads API study

### Import setting

In [1]:
import http.client, urllib.request, urllib.parse, urllib.error, base64
from google.transit import gtfs_realtime_pb2
import pandas as pd
import folium
from folium.plugins import BeautifyIcon
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import datetime

In [2]:
headers = {'Ocp-Apim-Subscription-Key': '46f44fa970e44e04be413233229d3c09',}

params = urllib.parse.urlencode({
})

feed = gtfs_realtime_pb2.FeedMessage()

## Vicroads planned closure API

### Services Alerts
Cancellation of metro train trips

In [14]:
try:
    conn = http.client.HTTPSConnection('data-exchange-api.vicroads.vic.gov.au')
    conn.request("GET", "/opendata/v1/gtfsr/metrotrain-servicealerts?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    feed.ParseFromString(data)
    print(feed)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

header {
  gtfs_realtime_version: "2.0"
  incrementality: FULL_DATASET
  timestamp: 1652593410
}



### Trip Updates
The trip update feed provides real-time arrival and departure information of a trip where data is available.

In [17]:
try:
    conn = http.client.HTTPSConnection('data-exchange-api.vicroads.vic.gov.au')
    conn.request("GET", "/opendata/v1/gtfsr/metrotrain-tripupdates?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    feed.ParseFromString(data)
    print(feed)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

header {
  gtfs_realtime_version: "2.0"
  incrementality: FULL_DATASET
  timestamp: 1652593440
}
entity {
  id: "2022-05-15-6329"
  trip_update {
    trip {
      trip_id: "100.UJ.2-WMN-B-mjp-1.2.H"
      start_time: "15:16:00"
      start_date: "20220515"
    }
    stop_time_update {
      stop_sequence: 5
      arrival {
        time: 1652592540
      }
      departure {
        time: 1652592600
      }
    }
    stop_time_update {
      stop_sequence: 6
      arrival {
        time: 1652592660
      }
      departure {
        time: 1652592660
      }
    }
    stop_time_update {
      stop_sequence: 7
      arrival {
        time: 1652592780
      }
      departure {
        time: 1652592780
      }
    }
    stop_time_update {
      stop_sequence: 8
      arrival {
        time: 1652592900
      }
      departure {
        time: 1652592960
      }
    }
    stop_time_update {
      stop_sequence: 9
      arrival {
        time: 1652593020
      }
      departure {
        time: 16

In [18]:
trip_list = []
stop_time_list = []

for entity in feed.entity:
  if entity.HasField('trip_update'):
      trip_update = entity.trip_update.trip
      trip_list.append(
          {'trip_id': trip_update.trip_id,
           'start_time': trip_update.start_time,
           'start_date': trip_update.start_date
           })

      stop_times = entity.trip_update.stop_time_update
      for st in stop_times:
        stop_time_list.append(
          {'trip_id': trip_update.trip_id,
           'stop_sequence': st.stop_sequence,
           'arrival_time': st.arrival.time,
           'depart_time': st.departure.time,
           })


df_trip_update = pd.DataFrame(trip_list, columns = ['trip_id','start_time','start_date'])
df_stop_time_update = pd.DataFrame(stop_time_list, columns = ['trip_id','stop_sequence','arrival_time','depart_time'])

# Clean data
df_trip_update['start_date'] = pd.to_datetime(df_trip_update['start_date'], format='%Y%m%d')
df_stop_time_update['arrival_time'] = pd.to_datetime(df_stop_time_update['arrival_time'], unit='s')
df_stop_time_update['depart_time'] = pd.to_datetime(df_stop_time_update['depart_time'], unit='s')

display(df_trip_update)
display(df_stop_time_update.head(20))

,trip_id,start_time,start_date
0,100.UJ.2-WMN-B-mjp-1.2.H,15:16:00,2022-05-15
1,105.UJ.2-WMN-B-mjp-1.4.R,16:16:00,2022-05-15
2,107.UJ.2-WMN-B-mjp-1.2.H,15:36:00,2022-05-15
3,1127.UJ.2-SYM-C-mjp-1.21.R,15:27:00,2022-05-15
4,1129.UJ.2-SYM-C-mjp-1.21.R,16:07:00,2022-05-15
...,...,...,...
177,9625.UJ.2-SYM-C-mjp-1.3.H,16:22:00,2022-05-15
178,965.UJ.2-SYM-C-mjp-1.19.R,16:56:00,2022-05-15
179,9793.UJ.2-SYM-C-mjp-1.5.H,15:22:00,2022-05-15
180,9797.UJ.2-SYM-C-mjp-1.5.H,16:02:00,2022-05-15


,trip_id,stop_sequence,arrival_time,depart_time
0,100.UJ.2-WMN-B-mjp-1.2.H,5,2022-05-15 05:29:00,2022-05-15 05:30:00
1,100.UJ.2-WMN-B-mjp-1.2.H,6,2022-05-15 05:31:00,2022-05-15 05:31:00
2,100.UJ.2-WMN-B-mjp-1.2.H,7,2022-05-15 05:33:00,2022-05-15 05:33:00
3,100.UJ.2-WMN-B-mjp-1.2.H,8,2022-05-15 05:35:00,2022-05-15 05:36:00
4,100.UJ.2-WMN-B-mjp-1.2.H,9,2022-05-15 05:37:00,2022-05-15 05:38:00
5,100.UJ.2-WMN-B-mjp-1.2.H,10,2022-05-15 05:40:00,2022-05-15 05:40:00
6,100.UJ.2-WMN-B-mjp-1.2.H,11,2022-05-15 05:42:00,2022-05-15 05:42:00
7,105.UJ.2-WMN-B-mjp-1.4.R,1,2022-05-15 06:04:00,2022-05-15 06:16:00
8,105.UJ.2-WMN-B-mjp-1.4.R,2,2022-05-15 06:17:00,2022-05-15 06:18:00
9,105.UJ.2-WMN-B-mjp-1.4.R,3,2022-05-15 06:19:00,2022-05-15 06:20:00


### Vehicle Positions
The vehicle position feed contains live location and occupancy of the service.

In [10]:
try:
    conn = http.client.HTTPSConnection('data-exchange-api.vicroads.vic.gov.au')
    conn.request("GET", "/opendata/v1/gtfsr/metrotrain-vehicleposition-updates?%s" % params, "{body}", headers)
    response = conn.getresponse()
    data = response.read()
    feed.ParseFromString(data)
    print(feed)
    conn.close()
except Exception as e:
    print("[Errno {0}] {1}".format(e.errno, e.strerror))

header {
  gtfs_realtime_version: "2.0"
  incrementality: FULL_DATASET
  timestamp: 1652942017
}
entity {
  id: "2022-05-19-6711"
  vehicle {
    trip {
      trip_id: "1065.T5.2-SYM-D-mjp-1.9.H"
      start_time: "16:07:00"
      start_date: "20220519"
    }
    position {
      latitude: -37.7806282043457
      longitude: 144.8269805908203
      bearing: -26.901092529296875
    }
    timestamp: 1652941962
    vehicle {
      id: "2523T-2569T-745M-746M-837M-838M"
    }
    occupancy_status: FEW_SEATS_AVAILABLE
  }
}
entity {
  id: "2022-05-19-6713"
  vehicle {
    trip {
      trip_id: "1073.T5.2-SYM-D-mjp-1.9.H"
      start_time: "16:38:00"
      start_date: "20220519"
    }
    position {
      latitude: -37.820960998535156
      longitude: 144.9542694091797
      bearing: 123.9365005493164
    }
    timestamp: 1652941953
    vehicle {
      id: "1019T-1028T-337M-355M-356M-438M"
    }
    occupancy_status: FEW_SEATS_AVAILABLE
  }
}
entity {
  id: "2022-05-19-6715"
  vehicle {
    tr

In [11]:
location_list = []

for entity in feed.entity:
  if entity.HasField('vehicle'):
      trip_update = entity.vehicle.trip
      position = entity.vehicle.position
      location_list.append(
          {'trip_id': trip_update.trip_id,
           'start_time': trip_update.start_time,
           'start_date': trip_update.start_date,
           'lat': position.latitude,
           'lon': position.longitude,
           'bearing': position.bearing,
           'timestamp': entity.vehicle.timestamp,
           'vehicle_id': entity.vehicle.vehicle.id,
           'occupancy_stat': entity.vehicle.occupancy_status
           })

df_vehicle_location = pd.DataFrame(location_list, columns = ['trip_id', 'start_time', 'start_date', 'lat', 'lon', 'bearing', 'timestamp', 'vehicle_id', 'occupancy_stat'])

# Clean data
df_vehicle_location['start_date'] = pd.to_datetime(df_vehicle_location['start_date'], format='%Y%m%d')
df_vehicle_location['timestamp'] = pd.to_datetime(df_vehicle_location['timestamp'], unit='s')
df_vehicle_location['timestamp'] = df_vehicle_location['timestamp'].dt.tz_localize('UTC').dt.tz_convert('Australia/Sydney')

display(df_vehicle_location)

,trip_id,start_time,start_date,lat,lon,bearing,timestamp,vehicle_id,occupancy_stat
0,1065.T5.2-SYM-D-mjp-1.9.H,16:07:00,2022-05-19,-37.780628,144.826981,-26.901093,2022-05-19 16:32:42+10:00,2523T-2569T-745M-746M-837M-838M,2
1,1073.T5.2-SYM-D-mjp-1.9.H,16:38:00,2022-05-19,-37.820961,144.954269,123.936501,2022-05-19 16:32:33+10:00,1019T-1028T-337M-355M-356M-438M,2
2,1081.T5.2-SYM-D-mjp-1.11.H,16:50:00,2022-05-19,-37.775539,144.921967,152.020523,2022-05-19 16:32:25+10:00,2501T-2543T-701M-702M-785M-786M,2
3,1087.T5.2-SYM-D-mjp-1.11.H,17:11:00,2022-05-19,-37.663815,144.925522,187.821884,2022-05-19 16:32:34+10:00,1118T-1120T-535M-536M-539M-540M,1
4,1093.T5.2-SYM-D-mjp-1.11.H,17:27:00,2022-05-19,-37.602249,144.943176,17.735466,2022-05-19 16:29:11+10:00,1149T-1158T-597M-598M-615M-616M,2
...,...,...,...,...,...,...,...,...,...
240,9951.T5.2-WBE-C-mjp-1.9.R,17:16:00,2022-05-19,-37.809704,144.943802,-32.918537,2022-05-19 16:31:31+10:00,1148T-1172T-595M-596M-643M-644M,1
241,9953.T5.2-WBE-C-mjp-1.1.H,16:26:00,2022-05-19,-37.809704,144.943802,-32.918537,2022-05-19 16:31:31+10:00,1148T-1172T-595M-596M-643M-644M,2
242,9957.T5.2-WBE-C-mjp-1.1.H,16:47:00,2022-05-19,-37.855381,145.018250,-37.919228,2022-05-19 16:32:44+10:00,1430T-1681T-259M-260M-961M-962M,2
243,9961.T5.2-WBE-C-mjp-1.1.H,17:07:00,2022-05-19,-37.937405,145.036423,1.396926,2022-05-19 16:32:42+10:00,1322T-1632T-43M-44M-863M-864M,2


**Note on occupancy_stat**
0 : EMPTY
1 : MANY_SEATS_AVAILABLE
2 : FEW_SEATS_AVAILABLE
3 : STANDING_ROOM_ONLY
...

Ref: https://developers.google.com/transit/gtfs-realtime/reference#enum-occupancystatus

#### Visualise Data

In [12]:
# Prepare colour dictionary for occupancy level
keys = list(df_vehicle_location['occupancy_stat'].unique())
color_range = list(np.linspace(0, 1, len(keys), endpoint=False))
colors = [matplotlib.colors.to_hex(plt.cm.Reds(x)) for x in color_range]
color_dict_industry = dict(zip(keys, colors))

In [13]:
# Vehicle Occupancy Status
train_occp = folium.FeatureGroup(name="Train Occupancy Status",
                                show=True,)


for i in range(0,len(df_vehicle_location)):

  # styles = {
  #   'fill': True,
  #   'color': color_dict_industry[df_vehicle_location.iloc[i]['occupancy_stat']],
  #   'weight': 1.5,
  #   # 'fillOpacity': 1
  # }

  icon = BeautifyIcon(
    icon='arrow-up',
    background_color=color_dict_industry[df_vehicle_location.iloc[i]['occupancy_stat']],
    # icon_shape='marker',
    inner_icon_style=f'transform: rotate({df_vehicle_location.iloc[i]["bearing"]}deg);'
  )

  html=f"""
      <h5>{df_vehicle_location.iloc[i]['trip_id']}</h5>
      <p>Occupancy Status: {df_vehicle_location.iloc[i]['occupancy_stat']}</p>
      <p>Start Time: {df_vehicle_location.iloc[i]['start_time']}</p>
      """
  iframe = folium.IFrame(html=html, width=200, height=200)
  popup = folium.Popup(iframe, max_width=2650)

  folium.Marker(
    location=[df_vehicle_location.iloc[i]['lat'], df_vehicle_location.iloc[i]['lon']],
    popup=popup,
    # radius=float(df_vehicle_location.iloc[i]['occupancy_stat'])*100,
    icon=icon,
    # **styles
  ).add_to(train_occp)

In [14]:
map = folium.Map(location=[-37.813, 144.945], tiles="CartoDB dark_matter", zoom_start=13)

train_occp.add_to(map)
folium.LayerControl(collapsed=False).add_to(map)

# Show the map
map